# TPS0222 LGBM predict_proba


## Public Score improved by repeated Optuna tuning!
* V1: 0.94533
* V2: 0.94623
* V3: 0.94829
* V4: 0.94498
* V5: 0.94914
* V6: 0.95617
* V7: 0.96039
* V9: 0.95808
* V10: 0.95943

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
Name0=train['target'].unique()
Name=sorted(Name0)
print(len(Name))
print(Name)

In [ ]:
N=list(range(len(Name)))   
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
target = train['target'].map(normal_mapping)
data = train.drop(['target','row_id'],axis=1)

In [ ]:
columns=data.columns.to_list()
print(columns)

In [ ]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2, random_state=42)
    param =   {
        'num_leaves': trial.suggest_int('num_leaves', 200, 250),
        'objective': 'regression',  
        'max_depth': trial.suggest_int('max_depth', 20, 23),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.1, 0.25),
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-5, 1e-1),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-5, 1e-1),
        "bagging_freq":trial.suggest_int('bagging_freq', 2, 8),
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.6, 0.7),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.7, 0.73),
        "verbosity": trial.suggest_int('verbosity', 2, 8),
    }
    model = lgb.LGBMClassifier(**param)      
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# shows the evolution of the search
optuna.visualization.plot_slice(study)

In [ ]:
# parameter interactions on an interactive chart.
optuna.visualization.plot_contour(study, params=['num_leaves','max_depth','learning_rate'])

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial=study.best_trial.params
print(Best_trial)

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
print(sample.shape)

In [ ]:
preds = np.zeros((sample.shape[0],len(Name)))
kf = KFold(n_splits=5,random_state=48,shuffle=True)
for trn_idx, test_idx in kf.split(train[columns],target):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=target.iloc[trn_idx],target.iloc[test_idx]
    model = lgb.LGBMClassifier(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict_proba(test[columns])/kf.n_splits   ###### predict_proba
    rmse=mean_squared_error(y_val, model.predict(X_val),squared=False)
    print(rmse)

In [ ]:
model.get_params

In [ ]:
print(preds.shape)
print(preds[0])

In [ ]:
from tqdm import tqdm
subm = sample
for i in tqdm(range(len(preds))):
    value=np.argmax(preds[i])
    subm.loc[i,'target']=reverse_mapping[value]

In [ ]:
subm.to_csv('submission.csv',index=False)
subm